In [ ]:

# # example of creating a CNN with an inception module
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import Conv2D
# from keras.layers import MaxPooling2D
# from keras.layers.merge import concatenate
# from keras.utils.vis_utils import plot_model
 
# # function for creating a naive inception block
# def naive_inception_module(layer_in, f1, f2, f3):
# 	# 1x1 conv
# 	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
# 	# 3x3 conv
# 	conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
# 	# 5x5 conv
# 	conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
# 	# 3x3 max pooling
# 	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
# 	# concatenate filters, assumes filters/channels last
# 	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
# 	return layer_out
 
# # define model input
# visible = Input(shape=(256, 256, 3))
# # add inception module
# layer = naive_inception_module(visible, 64, 128, 32)
# # create model
# model = Model(inputs=visible, outputs=layer)
# # summarize model
# model.summary()
# # plot model architecture
# plot_model(model, show_shapes=True, to_file='naive_inception_module.png')

In [ ]:
import os
import cv2
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, cohen_kappa_score
from keras.models import Model
from keras import optimizers, applications
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
#import tensorflow_addons as tfa
#from tensorflow.keras.metrics import Metric
#from tensorflow_addons.utils.types import AcceptableDTypes, FloatTensorLike
from typeguard import typechecked
from typing import Optional
%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
N_CLASSES = train['diagnosis'].nunique()
train["id_code"] = train["id_code"].apply(lambda x: x + ".png")
test["id_code"] = test["id_code"].apply(lambda x: x + ".png")
train['diagnosis'] = train['diagnosis'].astype('str')

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255, 
                                 validation_split=0.2,
                                 horizontal_flip=True)

train_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",
    target_size=(224, 224),
    subset='training')

In [ ]:
valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train,
    directory="../input/aptos2019-blindness-detection/train_images/",
    x_col="id_code",
    y_col="diagnosis",
    batch_size=16,
    class_mode="categorical",    
    target_size=(224, 224),
    subset='validation')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(  
        dataframe=test,
        directory = "../input/aptos2019-blindness-detection/test_images/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=16,
        shuffle=False,
        class_mode=None)

In [ ]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import Input, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model


baseModel=InceptionResNetV2(include_top=False, input_tensor=Input(shape=(256, 256, 3)), weights='../input/inceptionresnetv2/inceptionresnetv2weightstfdimnotop.h5')

for layer in baseModel.layers:
    layer.trainable=False
    
transfer_model=baseModel.output
transfer_model=MaxPooling2D(pool_size=(4, 4))(transfer_model)
transfer_model=Flatten()(transfer_model)
transfer_model=Dense(64, activation='relu')(transfer_model)
transfer_model=Dropout(0.2)(transfer_model)
transfer_model=Dense(5, activation='softmax')(transfer_model)

model = Model(inputs=baseModel.input, outputs=transfer_model)
model.summary()

In [ ]:
#taken from old keras source code
def f1_score(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),  
      tf.keras.metrics.AUC(name='auc'),
        f1_score,]

In [ ]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 2,verbose = 1,factor = 0.8, min_lr = 1e-6)

mcp = ModelCheckpoint('ResNet152V2.h5')

es = EarlyStopping(verbose=1, patience=2)

In [ ]:
model.compile(optimizer='Adam', loss="categorical_crossentropy", metrics=METRICS)
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data=valid_generator,
                              validation_steps=STEP_SIZE_VALID,
                              epochs=10,
                              callbacks=[lrd,mcp,es])

In [ ]:
complete_datagen = ImageDataGenerator(rescale=1./255)
complete_generator = complete_datagen.flow_from_dataframe(  
        dataframe=train,
        directory = "../input/aptos2019-blindness-detection/train_images/",
        x_col="id_code",
        target_size=(224, 224),
        batch_size=1,
        shuffle=False,
        class_mode=None)

In [ ]:
STEP_SIZE_COMPLETE = complete_generator.n//complete_generator.batch_size


In [ ]:
train_preds = model.predict_generator(complete_generator, steps=STEP_SIZE_COMPLETE)

In [ ]:
train_preds = [np.argmax(pred) for pred in train_preds]

In [ ]:
# test_generator.reset()
# STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
# preds = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
# predictions = [np.argmax(pred) for pred in preds]
# predictions[:10]

In [ ]:
test_generator.reset()
STEP_SIZE_TEST = test_generator.n//test_generator.batch_size
preds = model.predict_generator(test_generator)
predictions = [np.argmax(pred) for pred in preds]
predictions[:10]

In [ ]:
model.save('./output/inception')

In [ ]:
labels = ['0 - No DR', '1 - Mild', '2 - Moderate', '3 - Severe', '4 - Proliferative DR']
cnf_matrix = confusion_matrix(train['diagnosis'].astype('int'), train_preds)
cnf_matrix_norm = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(cnf_matrix_norm, index=labels, columns=labels)
print(df_cm.describe().T)
plt.figure(figsize=(15, 8))
sns.heatmap(df_cm, annot=True, fmt='.2f')
plt.show()

In [ ]:
# print(len(predictions), len(filenames), len(preds), test_generator.n)
# test

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"id_code":filenames,
                      "diagnosis":predictions})
results['id_code'] = results['id_code'].map(lambda x: str(x)[:-4])
results.to_csv("submission.csv",index=False)